# Vectorized Sring Operations

## Introducting Pandas String Operations

In [45]:
import numpy as np
x = np.array([2, 3, 5, 7, 11, 13])
x*2

array([ 4,  6, 10, 14, 22, 26])

In [46]:
data = ['Petter', "Paul", 'MARY', 'gUIDO']
[s.capitalize() for s in data]

['Petter', 'Paul', 'Mary', 'Guido']

In [47]:
data = ['Petter', "Paul", None, 'MARY', 'gUIDO']
[s.capitalize() for s in data]

AttributeError: 'NoneType' object has no attribute 'capitalize'

In [48]:
import pandas as pd
names = pd.Series(data)
names

0    Petter
1      Paul
2      None
3      MARY
4     gUIDO
dtype: object

In [5]:
names.str.capitalize()

0    Petter
1      Paul
2      None
3      Mary
4     Guido
dtype: object

## Tables of Pandas String Methods

In [6]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam', 
                  'Eric Idle', 'Terry Jones', 'Michael Palin'])

In [7]:
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

In [8]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

In [9]:
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

In [10]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

## Methods using regular expressions

In [15]:
monte.str.extract('([A-Za-z]+)',expand=False)

0     Graham
1       John
2      Terry
3       Eric
4      Terry
5    Michael
dtype: object

In [16]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

## Miscellaneous methods

### Vectorized item access and slicing

In [17]:
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

In [18]:
monte.str.split().str.get(-1)

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

### Indicator varaibles

In [19]:
full_monte = pd.DataFrame({'name': monte,
                          'info': ['B|C|D', 'B|D', 'A|C',
                                  'B|D', 'B|C', 'B|C|D']})
full_monte

,name,info
0,Graham Chapman,B|C|D
1,John Cleese,B|D
2,Terry Gilliam,A|C
3,Eric Idle,B|D
4,Terry Jones,B|C
5,Michael Palin,B|C|D


In [20]:
full_monte['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


# Example: Recipe Database

In [21]:
#!curl -O https://s3.amazonaws.com/openrecipes/20170107-061401-recipeitems.json.gz
#!gunzip 20170107-061401-recipeitems.json.gz 

In [24]:
try:
    recipes = pd.read_json('20170107-061401-recipeitems.json')
except ValueError as e:
    print(f'ValueError: {e}')

ValueError: Trailing data


In [27]:
#!pip install fsspec

In [49]:
from io import StringIO
#newdf = pd.read_json(StringIO(temp))

In [50]:
with open('20170107-061401-recipeitems.json') as f:
    line = f.readline()
pd.read_json(StringIO(line)).shape

(2, 12)

In [55]:
# read the entire file into a Python array
with open('20170107-061401-recipeitems.json','r') as f:
    # Extrat each line
    data = (line.strip() for line in f)
    # Reformat so each line is the element of a list
    data_json = "[{0}]".format(','.join(data))
# read the result as a JSON
recipes =  pd.read_json(StringIO(data_json))

In [56]:
recipes.shape

(173278, 17)

In [57]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
ts                                             {'$date': 1365276011104}
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
totalTime                                                           NaN
creator                                                         

In [58]:
recipes.columns

Index(['_id', 'name', 'ingredients', 'url', 'image', 'ts', 'cookTime',
       'source', 'recipeYield', 'datePublished', 'prepTime', 'description',
       'totalTime', 'creator', 'recipeCategory', 'dateModified',
       'recipeInstructions'],
      dtype='object')

In [59]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

In [60]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

In [61]:
recipes.description.str.contains('[Bd]reakfast').sum()

106

In [62]:
recipes.ingredients.str.contains('[Cc]inamon').sum()

11

### A simple recipe recommender

In [63]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley', 
             'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

In [68]:
import re 
spice_df = pd.DataFrame(dict((spice,recipes.ingredients.str.contains(spice,re.IGNORECASE))
                       for spice in spice_list))
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


In [69]:
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

10

In [70]:
recipes.name[selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object